In [35]:
from instagrapi import Client
from instagrapi.exceptions import LoginRequired, ClientJSONDecodeError
import config
import os

In [ ]:
# Setup Credentials
USERNAME = ""
PASSWORD = ""

In [36]:
# Setup client session and delay to avoid suspicion
cl = Client()
cl.delay_range = [1, 3]
SESSION_PATH = "session.json"

In [38]:
def login_user():

    session = cl.load_settings(SESSION_PATH)

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(USERNAME, PASSWORD)

            # Check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                print("Session is invalid, need to login via username and password")

                # Use the same device uuids across logins
                old_session = cl.get_settings()
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(USERNAME, PASSWORD)

            login_via_session = True
            cl.dump_settings(SESSION_PATH)
            print("Login via session successful")

        except Exception as e:
            print("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            print("Attempting to login via username and password. username: %s" % USERNAME)
            if cl.login(USERNAME, PASSWORD):
                login_via_pw = True
                cl.dump_settings(SESSION_PATH)
                print("Login via passoword is successful")

        except Exception as e:
            print("Couldn't login user using username and password: %s" % e)

    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")
    
login_user()

Login via session successful


In [39]:
# Input username to save
users = """
foo
bar
baz
"""

In [ ]:
def save_story(username):
    user_info = cl.user_info_by_username_v1("clausiee").model_dump()
    userid = user_info["pk"]
    print("Found user ID: "+userid)

    # Get story count
    stories = cl.user_stories(userid)
    print("User story count: "+str(len(stories)))

    # Save story if not exists
    for s in stories:
        storid = s.pk
        if not os.path.exists("stories"):
            os.makedirs("stories")
        if not (os.path.isfile(f"stories/{username}_{storid}.mp4") or os.path.isfile(f"stories/{username}_{storid}.jpg")):
            cl.story_download(storid, f"{username}_{storid}", "./stories")
            print(f"Story {storid} successfully downloaded!")
        else:
            print(f"Story {storid} already exists, skipping...")


for username in users.split():
    save_story(username)
    

Found user ID: 382503622
User story count: 4
Story 3556713366771380574 already exists, skipping...
Story 3556715267176887591 already exists, skipping...
Story 3556826321181627698 already exists, skipping...
Story 3556873093333572147 already exists, skipping...
